In [178]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import SnowballStemmer
from tensorboard.plugins.hparams import api as hp
import datetime
import tensorboard
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import LSTM , Dense ,Dropout , Input , Embedding , Bidirectional
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from wordcloud import WordCloud , STOPWORDS
import numpy as np
from keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
from tensorflow.keras.models import load_model 

In [10]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv("Train/train.csv")

In [11]:
df.sample(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
77591,cfe1130a315bc5e1,Knife in the back\n I came to you for help 3 times now... Is that something the bischo clan of puppets do? I think NOT! I actually started to like you and thought you were a nice guy sharing the music info but you know how to fool people. The knife i picked up (as you suggested in the investigation article) is the one you stuck in my back! I thought that maybe i found a decent person in you... \nThis community in wikipedia needs to learn to work together... and not always against everyone!,0,0,0,0,0,0
40370,6bb86d75cfd9a108,Thankyou. I honestly think that you should soon become an administrator. I would nominate you if i could and if you would like. BTW I think you should archive your talk page pretty soon:),0,0,0,0,0,0


## Trying Lemmatizer

In [3]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...


True

In [4]:
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

In [47]:
stop_words  = set(stopwords.words("english"))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [99]:
custom_patterns = [r"a+w+" , r"w+p+" , r"u+h+" , r"w+" , r"im"] 

In [141]:
custom_words = {
    "fuck": ["fucking" , "fuckin" , "f*$%-ing"] ,
    "dick" : ["dihck"],
    "nigga" : ["nihgaa", "nigger"],
    "ass" : ["arse"] 
}

In [123]:
def clean_sentence(sentence):
    sentence = sentence.lower()

    sentence = re.sub(r"\n"," ",sentence)
    sentence = re.sub(r"[( )]"," ",sentence)
    
    words = [word for word in sentence.split() if len(word)> 1]
    
    for i in range(len(words)):
        
        words[i] = wnl.lemmatize(words[i])
    sentence = " ".join(words)
    
    sentence = decontracted(sentence)

    for pattern in custom_patterns:
        sentence = re.sub(pattern, '', sentence)

    for word,variant in custom_words.items():
        for var in variant:
            sentence = str(sentence).replace(var,word)
            

    sentence = re.sub(r"[^a-zA-Z]"," ",sentence)
    
    sentence = re.sub("([^\x00-\x7F])+"," ",sentence)

    sentence = re.sub(r"\s+"," ",sentence)
    
    sentence = sentence.strip()

    # words = sentence.split()
    

    return sentence

In [86]:
def decontracted(phrase):
    # specific
    phrase = [word for word in phrase.split() if word not in stop_words]
    
    phrase = " ".join(phrase)
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"weren\'t" , "were not",phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase


In [147]:
print(clean_sentence(df[df["obscene"]==1]["comment_text"][141267]))


gay nigga god damndd gay nigga fuck fuck shtay ay kcik talk page bitch angry baby plz po anymore atalk dicuser page going cum ur house kill faggity


In [148]:
df.drop(columns=["id"],inplace=True)

In [150]:
df_copy = df.copy()
df_copy["comment_text"] = df_copy["comment_text"].apply(lambda x : clean_sentence(x))

In [153]:
df_copy["comment_text"].sample(5)

2157                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## Glove Embeddings

In [156]:
target_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']

In [157]:
X = df_copy["comment_text"].values
y = df_copy[target_cols].values

In [159]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

In [160]:
x_sequences = tokenizer.texts_to_sequences(X)
x_sequences[0]

[512,
 60,
 68,
 569,
 4042,
 10216,
 706,
 250,
 16409,
 5832,
 2366,
 2659,
 52,
 1076,
 11541,
 2508,
 11,
 160,
 209,
 7,
 4,
 80,
 6,
 19,
 3111,
 81]

In [169]:
x_padded_sequences = pad_sequences(x_sequences,maxlen=200,padding="post")

In [162]:
vocab_length = len(tokenizer.word_index )+1

In [166]:
glove_file = 'glove.6B.300d.txt'

embeddings_index = {}

with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]  
        coefs = np.asarray(values[1:], dtype='float32')  
        embeddings_index[word] = coefs  

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [167]:
embedding_matrix = np.zeros((vocab_length, 300))

for word,i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    # print(embedding_vector)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## LSTM Model

In [175]:
x_padded_sequences.shape

(159571, 200)

In [188]:
def build_Lstm(x_train,y_train,x_val,y_val,epochs,batch_size,vocab_size):

    input_layer = Input(shape=(200,),name="input layer")

    embed_layer = Embedding(vocab_size,
                            weights=[embedding_matrix],
                            trainable=False,
                            output_dim=300,
                            name="embeddings")
    
    embedded_seq = embed_layer(input_layer)

    x = Bidirectional(LSTM(units=64,return_sequences=True,activation="tanh" , name="lstm_1"))(embedded_seq)

    x = Dropout(rate=0.3 ,  name="dropout_layer")(x)

    x = Bidirectional(LSTM(units=32,return_sequences=False,activation="tanh",name="lstm_2"))(x)

    output_layer = Dense(units=6 , activation="sigmoid")(x)

    model = Model(inputs=input_layer,outputs=output_layer)

    model.compile(loss="binary_crossentropy" , optimizer="adam",metrics = ["accuracy"])


    
    model.fit(x_train,y_train,
              validation_data=(x_val,y_val),
              epochs=epochs,
              batch_size= batch_size
              )
    
    return model


In [180]:
x_train , x_test , y_train , y_test = train_test_split(x_padded_sequences,y , test_size=0.2,random_state=42)

In [181]:
x_train2 , x_val , y_train2 , y_val = train_test_split(x_train,y_train , test_size=0.2,random_state=42)

In [189]:
model = build_Lstm(x_train2,
                   y_train2,
                   x_val,
                   y_val,
                   epochs=3,
                   batch_size=32,
                   vocab_size=vocab_length)

Epoch 1/3
3192/3192 ━━━━━━━━━━━━━━━━━━━━ 906s 282ms/step - accuracy: 0.9460 - loss: 0.0964 - val_accuracy: 0.9939 - val_loss: 0.0571
Epoch 2/3
3192/3192 ━━━━━━━━━━━━━━━━━━━━ 922s 289ms/step - accuracy: 0.9938 - loss: 0.0530 - val_accuracy: 0.9937 - val_loss: 0.0520
Epoch 3/3
3192/3192 ━━━━━━━━━━━━━━━━━━━━ 1008s 316ms/step - accuracy: 0.9910 - loss: 0.0468 - val_accuracy: 0.9938 - val_loss: 0.0503


In [190]:
model.evaluate(x_test,y_test)

998/998 ━━━━━━━━━━━━━━━━━━━━ 126s 126ms/step - accuracy: 0.9939 - loss: 0.0473


[0.048964209854602814, 0.9937019944190979]

In [191]:
predictions = model.predict(x_test)

998/998 ━━━━━━━━━━━━━━━━━━━━ 103s 103ms/step


In [192]:
predictions[0]

array([0.29116   , 0.00138616, 0.01526344, 0.00879377, 0.0567487 ,
       0.01017578], dtype=float32)

In [201]:
y_test

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [204]:
predictions = (predictions > (0.5)).astype(int)

## Testing the Model

In [216]:
text = "you fucking bitch"

text = clean_sentence(text)
text

'fuck bitch'

In [217]:
test_sequence = tokenizer.texts_to_sequences(np.array([text]))
test_sequence

[[32, 514]]

In [218]:
padded_test_sequence = pad_sequences(test_sequence,200,padding="post")
test_predictions = model.predict(padded_test_sequence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [219]:
(test_predictions > (0.5)).astype(int)

array([[1, 0, 1, 0, 1, 0]])

In [220]:
target_cols

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

## Saving model , Tokenizer & Embedding Matrix

In [221]:
model.save("ToxicModelV2.h5")

In [224]:
import pickle
with open("tokenizer.pkl","wb") as file :
    pickle.dump(tokenizer,file)

In [228]:
import pickle
with open("embedding_matrix.pkl","wb") as file :
    pickle.dump(embedding_matrix,file)